In [ ]:
import json
import logging
import time
from datetime import datetime
from typing import Any, Dict, List

import requests
from dateutil.relativedelta import relativedelta
from pyspark.sql import SparkSession

In [2]:
minio_connection = ""  

In [3]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [4]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .master(master_url)
        )

        return builder.getOrCreate()

In [5]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/28 16:07:29 WARN Utils: Your hostname, DESKTOP-EDEM2DH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/28 16:07:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/edcarlos/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/edcarlos/.ivy2.5.2/cache
The jars for the packages stored in: /home/edcarlos/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de53d508-e17e-447d-9496-fde6253e1bdc;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central

In [6]:
# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [7]:
def fetch_ibge_ipca_amplo_data(url: str) -> List[Dict[str, Any]] | None:
    """
    Realiza a requisição HTTP e valida a resposta, com tratamento de erros e logs.
    """
    try:
        logging.info(f"Tentando requisição para: {url}")
        response = requests.get(url) 

    except requests.exceptions.ConnectionError:
        logging.error(f"Erro de conexão ao acessar {url}. Verifique sua rede.")
        return None
    except requests.exceptions.HTTPError as e:
        logging.error(f"Erro HTTP na requisição para {url}: {e}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro inesperado na requisição para {url}: {e}")
        return None
    
    content_type = response.headers.get('Content-Type', '')
    if 'application/json' not in content_type:
        logging.error(f" Resposta não é JSON: {content_type}")
        return None
        
    if len(response.content) == 0:
        logging.error(" Resposta vazia")
        return None
        
    logging.info("Requisição OK.")
    return response.json()

In [8]:
# Parâmetros fixos
agregados = 7060
variaveis = "63|69|2265|66"
localidades = "N1[all]|N6[all]"
classificacao = "315[all]"

In [9]:
# Gerar períodos do tipo "YYYYMM" de 2020 até hoje
start = datetime(2020, 1, 1)
end = datetime.today()
periodos = [
    (start + relativedelta(months=i)).strftime("%Y%m")
    for i in range((end.year - start.year) * 12 + end.month - start.month + 1)
]

In [10]:
# Função para montar a URL
def montar_url(periodo):
    return (
        f"https://servicodados.ibge.gov.br/api/v3/agregados/{agregados}"
        f"/periodos/{periodo}"
        f"/variaveis/{variaveis}"
        f"?localidades={localidades}"
        f"&classificacao={classificacao}"
    )

In [11]:
# Coleta os dados
dados = []

for periodo in periodos:
    url = montar_url(periodo)
    
    try:
        resp = requests.get(url)
        if resp.status_code == 200 and resp.json():
            dados.append(resp.json())
    except Exception:
        pass  # erro ignorado

    time.sleep(0.5)

    # Resultado: lista de JSONs por período


In [12]:
# Junta todos os resultados em um único response
response = []

for parcial in dados:
    if isinstance(parcial, list):
        response.extend(parcial)  # adiciona cada dicionário da lista parcial

In [13]:
if response:
    total_registros = len(response)
    exemplo = []

    for i, item in enumerate(response[:1], start=1): 
        registro = {k: v for k, v in item.items()}
        exemplo.append(registro)
else:
    total_registros = 0
    exemplo = []


In [14]:
"""
Navega por um JSON estruturado em: resultados → classificacoes + series

Faz joins manuais entre:

classificações ↔ categorias

localidades ↔ níveis

séries ↔ períodos e valores
"""

def safe_float(valor):
    try:
        return float(valor)
    except (ValueError, TypeError):
        return None

registros_extraidos = []

for resposta in response:
    for resultado in resposta['resultados']:
        classificacoes = resultado['classificacoes']
        series_temporais = resultado['series']

# Para cada classificação, iteramos todas as séries temporais
# para gerar todas as combinações possíveis (produto cartesiano)

        for classificacao in classificacoes:
            classificacao_copia = classificacao.copy()
            categorias = classificacao_copia.pop('categoria')

            for categoria_id, categoria_nome in categorias.items():
                classificacao_id = classificacao_copia['id']
                classificacao_nome = classificacao_copia['nome']

                for serie_temporal in series_temporais:
                    localidade = serie_temporal['localidade']
                    serie = serie_temporal['serie']

                    localidade_copia = localidade.copy()
                    nivel_info = localidade_copia.pop('nivel')

                    localidade_detalhada = {
                        **localidade_copia,
                        'nivel_id': nivel_info['id'],
                        'nivel_nome': nivel_info['nome']
                    }

                    for periodo, valor in serie.items():
                        registro = {
                            "id_variavel": resposta["id"],
                            "nome_variavel": resposta["variavel"],
                            "unidade_medida": resposta["unidade"],
                            "id_classificacao": classificacao_id,
                            "nome_classificacao": classificacao_nome,
                            "id_categoria": categoria_id,
                            "nome_categoria": categoria_nome,
                            "id_localidade": localidade_detalhada['id'],
                            "nome_localidade": localidade_detalhada['nome'],
                            "id_nivel": localidade_detalhada['nivel_id'],
                            "nome_nivel": localidade_detalhada['nivel_nome'],
                            "periodo": periodo,
                            "valor": safe_float(valor)
                        }
                        registros_extraidos.append(registro)



In [15]:
df = spark.createDataFrame(registros_extraidos)

In [16]:
df.write.format("delta").mode("overwrite").save("s3a://landing/ibge/ipca_amplo")

25/07/28 16:09:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
25/07/28 16:09:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/28 16:09:27 WARN TaskSetManager: Stage 6 contains a task of very large size (9086 KiB). The maximum recommended task size is 1000 KiB.
25/07/28 16:09:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/28 16:09:32 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to ibge/ipca_amplo/part-00005-ae7b59cd-301f-4a4f-9b51-c25

In [ ]:
spark.stop()

+------------+----------------+-------------+--------+-----------+--------------------+--------------------+-----------------+----------+--------------------+-------+--------------+-----+
|id_categoria|id_classificacao|id_localidade|id_nivel|id_variavel|      nome_categoria|  nome_classificacao|  nome_localidade|nome_nivel|       nome_variavel|periodo|unidade_medida|valor|
+------------+----------------+-------------+--------+-----------+--------------------+--------------------+-----------------+----------+--------------------+-------+--------------+-----+
|      107674|             315|      5300108|      N6|         69|8101006.Pós-gradu...|Geral, grupo, sub...|    Brasília - DF| Município|IPCA - Variação a...| 202105|             %| 0.53|
|       47664|             315|            1|      N1|         69|8101008.Educação ...|Geral, grupo, sub...|           Brasil|    Brasil|IPCA - Variação a...| 202105|             %|  1.1|
|       47664|             315|      1200401|      N6|      